In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                   transforms.Pad(4)]),
)

# Download test data from open datasets.
val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor(),
                                   transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                                    transforms.Pad(4)]),
)

100%|███████████████████████████████████████████████████████████████| 344862509/344862509 [02:34<00:00, 2228190.88it/s]


Extracting data\flowers-102\102flowers.tgz to data\flowers-102


100%|████████████████████████████████████████████████████████████████████████████████████████| 502/502 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 14989/14989 [00:00<?, ?it/s]


In [3]:
batch_size = 30
num_classes = 102
learning_rate = 0.001
num_epochs = 100

# Create data loaders.
#train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
val_dataloader = DataLoader(val_data, batch_size=batch_size)

In [4]:
import torch.nn.functional as F
import os

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.batch_layer1 = nn.BatchNorm2d(num_features=128) 
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=128, out_channels=512, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.batch_layer2 = nn.BatchNorm2d(num_features=512)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.conv_layer5 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.conv_layer6 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3)
        self.batch_layer3 = nn.BatchNorm2d(num_features=512)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(12800 , 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    

    def forward(self, x):
        out = self.relu1(self.conv_layer1(x))
        out = self.relu1(self.conv_layer2(out))
        out = self.batch_layer1(out)
        out = self.max_pool1(out)
        
        out = self.relu1(self.conv_layer3(out))
        out = self.relu1(self.conv_layer4(out))
        out = self.batch_layer2(out)
        out = self.max_pool2(out)

        out = self.relu1(self.conv_layer5(out))
        out = self.relu1(self.conv_layer6(out))
        out = self.batch_layer3(out)
        out = self.max_pool3(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

net = Net(num_classes)


In [5]:
net.load_state_dict(torch.load("net_29.pth"))
net.eval()

Net(
  (conv_layer1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch_layer1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer3): Conv2d(128, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer4): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (batch_layer2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer5): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (conv_layer6): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1))
  (batch_layer3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 1020 train images: 43.627450980392155 %


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))